<a href="https://colab.research.google.com/github/gulshan911/real-time-electricity-demand-forecasting/blob/main/real_time_electricity_demand_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install fastapi uvicorn pandas numpy pyngrok nest-asyncio plotly websockets

# Import libraries
import numpy as np
import pandas as pd
from datetime import datetime
import time
import asyncio
from fastapi import FastAPI, WebSocket
from fastapi.responses import HTMLResponse
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import threading
import plotly.graph_objects as go
import json

# Configure ngrok
ngrok.set_auth_token("2wHMjkerM4zOd0W2Vz36IcmLK0h_4J72XmqyQb1hjYEUhn1dd")  # <-- Put your token here

# ======================[ DATA LOADING ]======================
# Load your dataset here
# Option 1: Upload a CSV to Colab
from google.colab import files
uploaded = files.upload()

# Option 2: Load it
df = pd.read_csv(list(uploaded.keys())[0])

# Print to check
print(df.head())

# Required Columns: timestamp, demand, temperature
# If timestamp is missing, generate fake timestamps
if 'timestamp' not in df.columns:
    df['timestamp'] = pd.date_range(start=datetime.now(), periods=len(df), freq='min')
else:
    df['timestamp'] = pd.to_datetime(df['timestamp'])

# Optional: Fill missing columns
if 'demand' not in df.columns:
    df['demand'] = np.random.randint(1500, 3000, size=len(df))
if 'temperature' not in df.columns:
    df['temperature'] = np.random.uniform(15, 30, size=len(df))
if 'humidity' not in df.columns:
    df['humidity'] = np.random.uniform(40, 70, size=len(df))

# ======================[ BACKEND ]======================
class DatasetStreamer:
    def __init__(self, dataframe): # Changed _init_ to __init__
        self.dataframe = dataframe
        self.index = 0

    def next_sample(self):
        """Get next sample from dataset"""
        if self.index >= len(self.dataframe):
            self.index = 0  # loop again if reached end

        row = self.dataframe.iloc[self.index]
        self.index += 1

        record = {
            'timestamp': row['timestamp'].strftime('%H:%M:%S'),
            'demand': float(row['demand']),
            'temperature': float(row['temperature']),
            'humidity': float(row['humidity'])
        }
        return record

# Initialize components
data_stream = DatasetStreamer(df)

# FastAPI App
app = FastAPI()

@app.get("/", response_class=HTMLResponse)
async def get():
    return """
    <!DOCTYPE html>
    <html>
    <head>
        <title> Real-Time Electricity Demand Forecasting for Smart Cities Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    </head>
    <body>
        <h1>Real-Time Electricity Demand Forecasting for Smart Cities Dashboard </h1>
        <div id="graph" style="width:100%;height:600px;"></div>

        <script>
            var socket = new WebSocket("wss://" + window.location.host + "/ws");

            var timestamps = [];
            var demand = [];
            var temperature = [];

            var layout = {
                title: "Electricity Demand and Temperature",
                xaxis: {title: "Time"},
                yaxis: {title: "Values"},
            };

            Plotly.newPlot('graph', [
                {x: timestamps, y: demand, name: "Demand (MW)", type: 'scatter'},
                {x: timestamps, y: temperature, name: "Temperature (°C)", type: 'scatter'}
            ], layout);

            socket.onmessage = function(event) {
                var data = JSON.parse(event.data);
                timestamps.push(data.timestamp);
                demand.push(data.demand);
                temperature.push(data.temperature);

                Plotly.update('graph', {
                    x: [timestamps, timestamps],
                    y: [demand, temperature]
                });
            };
        </script>
    </body>
    </html>
    """

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        latest = data_stream.next_sample()
        await websocket.send_text(json.dumps(latest))
        await asyncio.sleep(2)  # Send data every 2 seconds

# ======================[ RUN EVERYTHING ]======================
def run():
    nest_asyncio.apply()

    public_url = ngrok.connect(8000, bind_tls=True)
    print(f" Public URL: {public_url}")
    print(f" Open Dashboard: {public_url}/")

    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the application
run()

Saving large_electricity_demand.csv to large_electricity_demand.csv
             timestamp      demand  temperature   humidity  day_of_week  hour
0  2010-01-01 00:00:00  549.671415    26.300459  68.822255            4     0
1  2010-01-01 01:00:00  486.173570    21.803272  39.802336            4     1
2  2010-01-01 02:00:00  564.768854    32.751799  80.277765            4     2
3  2010-01-01 03:00:00  652.302986    20.857642  55.948560            4     3
4  2010-01-01 04:00:00  476.584663    28.226984  57.169900            4     4


INFO:     Started server process [268]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


 Public URL: NgrokTunnel: "https://dd21-34-16-194-53.ngrok-free.app" -> "http://localhost:8000"
 Open Dashboard: NgrokTunnel: "https://dd21-34-16-194-53.ngrok-free.app" -> "http://localhost:8000"/
INFO:     42.104.249.16:0 - "GET / HTTP/1.1" 200 OK
INFO:     42.104.249.16:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     42.104.249.16:0 - "WebSocket /ws" [accepted]
INFO:     connection open
